## Load Data

In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

df_kbli_real = pd.read_csv('../dataset/clean/clean_sak_occupation_kalteng.csv',dtype={"kbli_code": str, "kbli_db": str, "kbji_code": str, "kbji_label": str})

df_kbli_real.head()

,text_description,kbli_code,kbli_label,kbji_code,kbji_label
0,Aktivitas: Karyawan panen kelapa sawit di kebu...,01262,PERKEBUNAN BUAH KELAPA SAWIT,6113,PEKERJA PERTANIAN TANAMAN KEBUN BIBIT DAN TANA...
1,"Aktivitas: Karyawan perawatan (memupuk, nyempr...",01262,PERKEBUNAN BUAH KELAPA SAWIT,6113,PEKERJA PERTANIAN TANAMAN KEBUN BIBIT DAN TANA...
2,Aktivitas: Ketua koperasi kelompok tani mitra ...,01262,PERKEBUNAN BUAH KELAPA SAWIT,1211,MANAJER KEUANGAN
3,"Aktivitas: Merawat, membersihkan, memupuk kebu...",01262,PERKEBUNAN BUAH KELAPA SAWIT,6112,PEKERJA PERTANIAN TANAMAN TAHUNAN DAN TANAMAN ...
4,Aktivitas: Pemilik dan penjual hasil kebun kel...,01262,PERKEBUNAN BUAH KELAPA SAWIT,6112,PEKERJA PERTANIAN TANAMAN TAHUNAN DAN TANAMAN ...


In [9]:
### load kbli labe description

kbli_label=pd.read_csv("../dataset/kbli2020.csv")

kbli_label.head()

,kategori,digit,kode,judul,deskripsi
0,A,1,A,"Pertanian, Kehutanan dan Perikanan",Kategori ini mencakup semua kegiatan ekonomi/l...
1,A,2,01,"Pertanian Tanaman, Peternakan, Perburuan dan K...",Golongan pokok ini mencakup pertanian tanaman ...
2,A,3,011,Pertanian Tanaman Semusim,Golongan ini mencakup penanaman tanaman yang t...
3,A,4,0111,"Pertanian serealia (bukan padi), aneka kacang ...",Subgolongan ini mencakup pertanian semua serea...
4,A,5,01111,Pertanian Jagung,Kelompok ini mencakup usaha pertanian komodita...


In [2]:
### load kbli labe description

kbji_label=pd.read_csv("../dataset/kbji2014.csv",dtype={"kode": str})

kbji_label.head()

,digit,kode,judul,deskripsi
0,1,0,Tentara Nasional Indonesia (TNI) dan Kepolisia...,Golongan pokok ini mencakup semua pekerjaan ya...
1,2,01,Perwira TNI dan POLRI,Perwira TNI dan POLRI memberikan kepemimpinan ...
2,3,011,Perwira TNI dan POLRI,Perwira TNI dan POLRI memberikan kepemimpinan ...
3,4,0111,Perwira Markas Besar TNI,Perwira Mabes TNI memberikan kepemimpinan dan ...
4,6,0111.01,Panglima TNI,Panglima TNI adalah pimpinan TNI yang berkedud...


## Define merge label function

In [10]:
def extract_hierarki_kbji(kbji_code,kbji_db):
    code = str(kbji_code["kbji_code"])
    #print(f"code:{code}")
    #kbji_label_df=kbji_db[kbji_db['kode']==code]
    #print(f"kbji_kategori:{kbji_label_df['kategori'].values[0]}")
    kategori=code[:1]
    kbji_code_2=code[:2]
    kbji_code_3=code[:3]
    return kategori,kbji_code_2, kbji_code_3, code 


def merge_label(df,kbli_db,kbji_db): 
    temp_hicode = df.apply(lambda row: extract_hierarki_kbji(row, kbji_db), axis=1)
    temp_hicode = pd.DataFrame(temp_hicode.tolist(), columns=['kbji_kategori', 'kbji_code_2', 'kbji_code_3', 'kbji_code_4'])
    df_hierarki_kbji = pd.concat([df, temp_hicode], axis=1)
    df_hierarki_kbji = df_hierarki_kbji.merge(kbji_db[kbji_db["digit"]==1][["kode","judul","deskripsi"]], 
    left_on='kbji_kategori', right_on='kode', how='left').rename(columns={"deskripsi": "kbji_kategori_desc",
     "judul":"kbji_kategori_title"}).drop(columns="kode")

     # 2. kbji 
    df_hierarki_kbji = df_hierarki_kbji.merge(kbji_db[kbji_db["digit"]==2][["kode","judul","deskripsi"]], 
    left_on='kbji_code_2', right_on='kode', how='left').rename(columns={"deskripsi": "kbji_code_2_desc",
     "judul":"kbji_code_2_title"}).drop(columns="kode")

    df_hierarki_kbji = df_hierarki_kbji.merge(kbji_db[kbji_db["digit"]==3][["kode","judul","deskripsi"]], 
    left_on='kbji_code_3', right_on='kode', how='left').rename(columns={"deskripsi": "kbji_code_3_desc",
     "judul":"kbji_code_3_title"}).drop(columns="kode")

    df_hierarki_kbji = df_hierarki_kbji.merge(kbji_db[kbji_db["digit"]==4][["kode","judul","deskripsi"]], 
    left_on='kbji_code_4', right_on='kode', how='left').rename(columns={"deskripsi": "kbji_code_4_desc",
     "judul":"kbji_code_4_title"}).drop(columns="kode")
    return df_hierarki_kbji

merge_label(df_kbli_real,kbli_label,kbji_label)


,text_description,kbli_code,kbli_label,kbji_code,kbji_label,kbji_kategori,kbji_code_2,kbji_code_3,kbji_code_4,kbji_kategori_title,kbji_kategori_desc,kbji_code_2_title,kbji_code_2_desc,kbji_code_3_title,kbji_code_3_desc,kbji_code_4_title,kbji_code_4_desc
0,Aktivitas: Karyawan panen kelapa sawit di kebu...,01262,PERKEBUNAN BUAH KELAPA SAWIT,6113,PEKERJA PERTANIAN TANAMAN KEBUN BIBIT DAN TANA...,6,61,611,6113,"Pekerja Terampil Pertanian, Kehutanan dan Peri...",Pekerja ini menanam dan memanen tanaman atau p...,Pekerja Terampil Pertanian Berorientasi Pasar,"Pekerja ini merencanakan, mengatur dan melakuk...",Pekerja Pertanian dan Perkebunan,"Pekerja ini merencanakan, mengatur dan melakuk...",Pekerja Pertanian Tanaman Kebun Bibit dan Tana...,"Pekerja ini merencanakan, mengatur dan melakuk..."
1,"Aktivitas: Karyawan perawatan (memupuk, nyempr...",01262,PERKEBUNAN BUAH KELAPA SAWIT,6113,PEKERJA PERTANIAN TANAMAN KEBUN BIBIT DAN TANA...,6,61,611,6113,"Pekerja Terampil Pertanian, Kehutanan dan Peri...",Pekerja ini menanam dan memanen tanaman atau p...,Pekerja Terampil Pertanian Berorientasi Pasar,"Pekerja ini merencanakan, mengatur dan melakuk...",Pekerja Pertanian dan Perkebunan,"Pekerja ini merencanakan, mengatur dan melakuk...",Pekerja Pertanian Tanaman Kebun Bibit dan Tana...,"Pekerja ini merencanakan, mengatur dan melakuk..."
2,Aktivitas: Ketua koperasi kelompok tani mitra ...,01262,PERKEBUNAN BUAH KELAPA SAWIT,1211,MANAJER KEUANGAN,1,12,121,1211,Manajer,"Manajer merencanakan, mengarahkan, mengkoordin...",Manajer Administrasi dan Komersial,Manajer administrasi dan komersial merencanaka...,Manajer Pelayanan Bisnis dan Administrasi Bisnis,Manajer pelayanan bisnis dan administrasi bisn...,Manajer Keuangan,"Manajer keuangan merencanakan, mengarahkan dan..."
3,"Aktivitas: Merawat, membersihkan, memupuk kebu...",01262,PERKEBUNAN BUAH KELAPA SAWIT,6112,PEKERJA PERTANIAN TANAMAN TAHUNAN DAN TANAMAN ...,6,61,611,6112,"Pekerja Terampil Pertanian, Kehutanan dan Peri...",Pekerja ini menanam dan memanen tanaman atau p...,Pekerja Terampil Pertanian Berorientasi Pasar,"Pekerja ini merencanakan, mengatur dan melakuk...",Pekerja Pertanian dan Perkebunan,"Pekerja ini merencanakan, mengatur dan melakuk...",Pekerja Pertanian Tanaman Tahunan dan Tanaman ...,Pekerja ini mengatur dan melakukan kegiatan pe...
4,Aktivitas: Pemilik dan penjual hasil kebun kel...,01262,PERKEBUNAN BUAH KELAPA SAWIT,6112,PEKERJA PERTANIAN TANAMAN TAHUNAN DAN TANAMAN ...,6,61,611,6112,"Pekerja Terampil Pertanian, Kehutanan dan Peri...",Pekerja ini menanam dan memanen tanaman atau p...,Pekerja Terampil Pertanian Berorientasi Pasar,"Pekerja ini merencanakan, mengatur dan melakuk...",Pekerja Pertanian dan Perkebunan,"Pekerja ini merencanakan, mengatur dan melakuk...",Pekerja Pertanian Tanaman Tahunan dan Tanaman ...,Pekerja ini mengatur dan melakukan kegiatan pe...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11737,Aktivitas: Menangkap dan memancing berbagai ma...,03125,PENANGKAPAN/PENGAMBILAN INDUK/BENIH IKAN DI PE...,6222,PEKERJA PERIKANAN TANGKAP DI PERAIRAN UMUM DAN...,6,62,622,6222,"Pekerja Terampil Pertanian, Kehutanan dan Peri...",Pekerja ini menanam dan memanen tanaman atau p...,"Pekerja Terampil Kehutanan, Perikanan dan Perb...","Pekerja ini merencanakan, mengatur dan melakuk...","Pekerja Perikanan, Perburuan, dan Penangkapan ...","Pekerja ini membiakkan dan membesarkan ikan, m...",Pekerja Perikanan Tangkap Di Perairan Umum dan...,Pekerja ini sendiri atau sebagai anggota kru k...
11738,Aktivitas: Menyedot pasir di sungai untuk menc...,07301,PERTAMBANGAN EMAS DAN PERAK,8111,PEKERJA TAMBANG DAN PEKERJA PENGGALIAN,8,81,811,8111,Operator dan Perakit Mesin,Operator dan perakit mesin mengoperasikan dan ...,Operator Mesin Stasioner dan YBDI,Operator mesin stasioner dan ybdi mengoperasik...,Operator Mesin Pengolahan Bahan Tambang dan Mi...,Operator mesin pengolah bahan tambang dan mine...,Pekerja Tambang dan Pekerja Penggalian,Pekerja tambang dan pekerja

In [16]:
def extract_hierarki_kbli(kbli_code,kbli_db):
    #code=kbli_code["kbli_code"]
    code = str(kbli_code["kbli_code"])
    kbli_label_df=kbli_db[kbli_db['kode']==code]
    
    #print(f"kbli_kategori:{kbli_db_5_df['kategori'].values[0]}")
    kategori=kbli_label_df['kategori'].values[0]
    # print(kbli_db_5_df)
    kbli_code_5=code[:5]
    kbli_code_4=code[:4]
    kbli_code_3=code[:3]
    kbli_code_2=code[:2]
    return kategori,kbli_code_2, kbli_code_3, kbli_code_4, code



def merge_label(df,kbli_db,kbji_db): 
    ##-- col df:text_description,kbli_code,kbji_code 
    ##-- col kbli_db: kategori,digit,kode,judul,deskripsi
    ##-- col kbji_db: kategori,digit,kode,judul,deskripsi
    temp_hicode = df.apply(lambda row: extract_hierarki_kbli(row, kbli_db), axis=1)
    temp_hicode=temp_hicode.apply(pd.Series)
    temp_hicode.columns=["kbli_code_1","kbli_code_2","kbli_code_3","kbli_code_4","kbli_code_5"]
    df_hierarki_kbli=pd.concat([df,temp_hicode],axis=1)
    #print(df_hierarki_kbli.columns)
    #1. kategori_desc
    df_hierarki_kbli = df_hierarki_kbli.merge(
    kbli_db[kbli_db["digit"] == 1][["kode","judul","deskripsi"]],
    left_on="kbli_code_1", right_on="kode",how="left").rename(columns={"deskripsi": "kbli_kategori_desc",
     "judul":"kbli_kategori_title"}).drop(columns="kode")
    # 2. kbli_code_2_desc
    df_hierarki_kbli = df_hierarki_kbli.merge(
        kbli_db[kbli_db["digit"] == 2][["kode","judul","deskripsi"]],
        left_on="kbli_code_2", right_on="kode", how="left"
    ).rename(columns={"deskripsi": "kbli_code_2_desc","judul":"kbli_code_2_title"}).drop(columns="kode")

    # 3. kbli_code_3_desc
    df_hierarki_kbli = df_hierarki_kbli.merge(
        kbli_db[kbli_db["digit"] == 3][["kode","judul","deskripsi"]],
        left_on="kbli_code_3", right_on="kode", how="left"
    ).rename(columns={"deskripsi": "kbli_code_3_desc","judul":"kbli_code_3_title"}).drop(columns="kode")

    # 4. kbli_code_4_desc
    df_hierarki_kbli = df_hierarki_kbli.merge(
        kbli_db[kbli_db["digit"] == 4][["kode","judul","deskripsi"]],
        left_on="kbli_code_4", right_on="kode", how="left"
    ).rename(columns={"deskripsi": "kbli_code_4_desc","judul":"kbli_code_4_title"}).drop(columns="kode")

    # 5. kbli_code_5_desc
    df_hierarki_kbli = df_hierarki_kbli.merge(
        kbli_db[kbli_db["digit"] == 5][["kode","judul","deskripsi"]],
        left_on="kbli_code_5", right_on="kode", how="left"
    ).rename(columns={"deskripsi": "kbli_code_5_desc","judul":"kbli_code_5_title"}).drop(columns="kode")

    temp_hicode = df.apply(lambda row: extract_hierarki_kbji(row, kbji_db), axis=1)
    temp_hicode = pd.DataFrame(temp_hicode.tolist(), columns=['kbji_code_1', 'kbji_code_2', 'kbji_code_3', 'kbji_code_4'])
    df_hierarki_kbji = pd.concat([df, temp_hicode], axis=1)
    df_hierarki_kbji = df_hierarki_kbji.merge(kbji_db[kbji_db["digit"]==1][["kode","judul","deskripsi"]], 
    left_on='kbji_code_1', right_on='kode', how='left').rename(columns={"deskripsi": "kbji_kategori_desc",
     "judul":"kbji_kategori_title"}).drop(columns="kode")


    df_hierarki_kbji = df_hierarki_kbji.merge(kbji_db[kbji_db["digit"]==2][["kode","judul","deskripsi"]], 
    left_on='kbji_code_2', right_on='kode', how='left').rename(columns={"deskripsi": "kbji_code_2_desc",
     "judul":"kbji_code_2_title"}).drop(columns="kode")

    df_hierarki_kbji = df_hierarki_kbji.merge(kbji_db[kbji_db["digit"]==3][["kode","judul","deskripsi"]], 
    left_on='kbji_code_3', right_on='kode', how='left').rename(columns={"deskripsi": "kbji_code_3_desc",
     "judul":"kbji_code_3_title"}).drop(columns="kode")

    df_hierarki_kbji = df_hierarki_kbji.merge(kbji_db[kbji_db["digit"]==4][["kode","judul","deskripsi"]], 
    left_on='kbji_code_4', right_on='kode', how='left').rename(columns={"deskripsi": "kbji_code_4_desc",
     "judul":"kbji_code_4_title"}).drop(columns="kode")

    #print(df_hierarki_kbji.columns)
     #combine df
    df_output=pd.concat([df_hierarki_kbli,df_hierarki_kbji[['kbji_code_1', 'kbji_code_2', 'kbji_code_3',
       'kbji_code_4', 'kbji_kategori_title', 'kbji_kategori_desc',
       'kbji_code_2_title', 'kbji_code_2_desc', 'kbji_code_3_title',
       'kbji_code_3_desc', 'kbji_code_4_title', 'kbji_code_4_desc']]],axis=1)
    return df_output


df_output=merge_label(df=df_kbli_real,kbli_db=kbli_label,kbji_db=kbji_label)

print(df_output.columns)

Index(['text_description', 'kbli_code', 'kbli_label', 'kbji_code',
       'kbji_label', 'kbli_code_1', 'kbli_code_2', 'kbli_code_3',
       'kbli_code_4', 'kbli_code_5', 'kbli_kategori_title',
       'kbli_kategori_desc', 'kbli_code_2_title', 'kbli_code_2_desc',
       'kbli_code_3_title', 'kbli_code_3_desc', 'kbli_code_4_title',
       'kbli_code_4_desc', 'kbli_code_5_title', 'kbli_code_5_desc',
       'kbji_code_1', 'kbji_code_2', 'kbji_code_3', 'kbji_code_4',
       'kbji_kategori_title', 'kbji_kategori_desc', 'kbji_code_2_title',
       'kbji_code_2_desc', 'kbji_code_3_title', 'kbji_code_3_desc',
       'kbji_code_4_title', 'kbji_code_4_desc'],
      dtype='object')


In [20]:
##export df_output to csv
import csv

df_output.to_csv('../dataset/clean/df_output_kalteng.csv',index=False, sep=',', quotechar='"', quoting=csv.QUOTE_ALL, encoding='utf-8')

In [21]:
## load clean data
import pandas as pd
import numpy as np
import re

df_loaded = pd.read_csv('../dataset/clean/df_output_kalteng.csv', 
                        sep=',', 
                        quotechar='"', 
                        encoding='utf-8')
df_loaded.head()

,text_description,kbli_code,kbli_label,kbji_code,kbji_label,kbli_code_1,kbli_code_2,kbli_code_3,kbli_code_4,kbli_code_5,...,kbji_code_3,kbji_code_4,kbji_kategori_title,kbji_kategori_desc,kbji_code_2_title,kbji_code_2_desc,kbji_code_3_title,kbji_code_3_desc,kbji_code_4_title,kbji_code_4_desc
0,Aktivitas: Karyawan panen kelapa sawit di kebu...,1262,PERKEBUNAN BUAH KELAPA SAWIT,6113,PEKERJA PERTANIAN TANAMAN KEBUN BIBIT DAN TANA...,A,1,12,126,1262,...,611,6113,"Pekerja Terampil Pertanian, Kehutanan dan Peri...",Pekerja ini menanam dan memanen tanaman atau p...,Pekerja Terampil Pertanian Berorientasi Pasar,"Pekerja ini merencanakan, mengatur dan melakuk...",Pekerja Pertanian dan Perkebunan,"Pekerja ini merencanakan, mengatur dan melakuk...",Pekerja Pertanian Tanaman Kebun Bibit dan Tana...,"Pekerja ini merencanakan, mengatur dan melakuk..."
1,"Aktivitas: Karyawan perawatan (memupuk, nyempr...",1262,PERKEBUNAN BUAH KELAPA SAWIT,6113,PEKERJA PERTANIAN TANAMAN KEBUN BIBIT DAN TANA...,A,1,12,126,1262,...,611,6113,"Pekerja Terampil Pertanian, Kehutanan dan Peri...",Pekerja ini menanam dan memanen tanaman atau p...,Pekerja Terampil Pertanian Berorientasi Pasar,"Pekerja ini merencanakan, mengatur dan melakuk...",Pekerja Pertanian dan Perkebunan,"Pekerja ini merencanakan, mengatur dan melakuk...",Pekerja Pertanian Tanaman Kebun Bibit dan Tana...,"Pekerja ini merencanakan, mengatur dan melakuk..."
2,Aktivitas: Ketua koperasi kelompok tani mitra ...,1262,PERKEBUNAN BUAH KELAPA SAWIT,1211,MANAJER KEUANGAN,A,1,12,126,1262,...,121,1211,Manajer,"Manajer merencanakan, mengarahkan, mengkoordin...",Manajer Administrasi dan Komersial,Manajer administrasi dan komersial merencanaka...,Manajer Pelayanan Bisnis dan Administrasi Bisnis,Manajer pelayanan bisnis dan administrasi bisn...,Manajer Keuangan,"Manajer keuangan merencanakan, mengarahkan dan..."
3,"Aktivitas: Merawat, membersihkan, memupuk kebu...",1262,PERKEBUNAN BUAH KELAPA SAWIT,6112,PEKERJA PERTANIAN TANAMAN TAHUNAN DAN TANAMAN ...,A,1,12,126,1262,...,611,6112,"Pekerja Terampil Pertanian, Kehutanan dan Peri...",Pekerja ini menanam dan memanen tanaman atau p...,Pekerja Terampil Pertanian Berorientasi Pasar,"Pekerja ini merencanakan, mengatur dan melakuk...",Pekerja Pertanian dan Perkebunan,"Pekerja ini merencanakan, mengatur dan melakuk...",Pekerja Pertanian Tanaman Tahunan dan Tanaman ...,Pekerja ini mengatur dan melakukan kegiatan pe...
4,Aktivitas: Pemilik dan penjual hasil kebun kel...,1262,PERKEBUNAN BUAH KELAPA SAWIT,6112,PEKERJA PERTANIAN TANAMAN TAHUNAN DAN TANAMAN ...,A,1,12,126,1262,...,611,6112,"Pekerja Terampil Pertanian, Kehutanan dan Peri...",Pekerja ini menanam dan memanen tanaman atau p...,Pekerja Terampil Pertanian Berorientasi Pasar,"Pekerja ini merencanakan, mengatur dan melakuk...",Pekerja Pertanian dan Perkebunan,"Pekerja ini merencanakan, mengatur dan melakuk...",Pekerja Pertanian Tanaman Tahunan dan Tanaman ...,Pekerja ini mengatur dan melakukan kegiatan pe...


In [23]:
df_loaded[:100].to_csv('../dataset/clean/df_output_kalteng_sample.csv',index=False, sep=',', quotechar='"', quoting=csv.QUOTE_ALL, encoding='utf-8')